# Combining the multiselect with map

In [1]:
import pandas as pd
import numpy as np

import bokeh.plotting
import bokeh.io
from bokeh.plotting import figure, show, row
from bokeh.models import ColumnDataSource, CustomJS, MultiChoice

from bokeh.tile_providers import CARTODBPOSITRON, get_provider

tile_provider = get_provider(CARTODBPOSITRON)
bokeh.io.output_notebook()

Loading BokehJS ...

In [21]:
# Helper fxn to set coords right
def mercator_creator(df, lat='lat', lon='lon'):
    # from https://www.youtube.com/watch?v=BojxegBh9_4
    
    k = 6378137
    df['x'] = k * np.radians(df[lon])
    df['y'] = np.log(np.tan((90 + df[lat]) * np.pi / 360)) * k
    
    return df

# Data

In [23]:
# Load in the metadata
ferm_locations = pd.read_csv("./data/ferm.csv")

# Split the Lat-Long column into two columns, Lat and Long
ferm_locations[['Lat', 'Long']] = ferm_locations['Lat-Long'].str.split(', ', 1, expand=True)

# Create a new dataframe only if the point has a valid Lat and Long
ferm_locations_final = ferm_locations[ferm_locations['Lat'].notna()].copy()

# Convert from a string, and put in columns called Latitude and Longitude
ferm_locations_final.loc[:,'Latitude1'] = pd.to_numeric(ferm_locations_final['Lat'],errors='coerce')
ferm_locations_final.loc[:,'Longitude1'] = pd.to_numeric(ferm_locations_final['Long'],errors='coerce')

# Add x, y for lat long
ferm_locations_final = mercator_creator(ferm_locations_final, lat='Latitude1', lon='Longitude1')

class_options = list(ferm_locations_final.Class.unique())

# Map


In [11]:
chosen_cds = bokeh.models.ColumnDataSource(
    {
        "x": ferm_locations_final["x"],
        "y": ferm_locations_final["y"],
        "color": ["#1f77b3"] * len(ferm_locations_final),
    }
)

In [13]:
plot = bokeh.plotting.figure(
    frame_height=400,
    frame_width=700,
    x_axis_label="Latitude",
    y_axis_label="Longitude",
    tools =['hover', 'wheel_zoom', 'pan', 'reset'],
    x_axis_type = 'mercator',
    y_axis_type = 'mercator',
    x_range = [ferm_locations_final.x.min()-100000, ferm_locations_final.x.max() + 100000],
    y_range = [ferm_locations_final.y.min()-100000, ferm_locations_final.y.max() + 100000]
    
)

unique_classes = ferm_locations_final.Class.unique()
name_dict={'name':[],'legend':[],'label':[]}

for Class in unique_classes:  
    source = ColumnDataSource(ferm_locations_final[ferm_locations_final['Class']==Class])
    name_glyph = plot.circle('x', 'y', source=source, legend_label=Class)
    name_dict['name'].append(name_glyph)
    name_dict['label'].append(Class)

In [14]:
name_legend_dict={'name':[]}
for label in range(len(unique_classes)):
    name_dict['legend'].append(plot.legend.items[label])

In [15]:
# Set up MultiChoice widget
initial_value = [unique_classes[0]]
options = list(unique_classes)

In [16]:
for i in range(len(options)):
    if name_dict['label'][i] in initial_value:
        name_dict['name'][i].visible = True;
        name_dict['legend'][i].visible = True;
    else:
        name_dict['name'][i].visible = False;
        name_dict['legend'][i].visible = False;   

In [17]:
multi_choice = MultiChoice(value=initial_value, options=options, max_items=12, title='Selection:')

In [18]:

callback = CustomJS(args=dict(name_dict=name_dict, multi_choice=multi_choice), code="""
var selected_vals = multi_choice.value;
var index_check = [];

for (var i = 0; i < name_dict['name'].length; i++) {
    index_check[i]=selected_vals.indexOf(name_dict['label'][i]);
        if ((index_check[i])>= 0) {
            name_dict['name'][i].visible = true;
            name_dict['legend'][i].visible = true;
            }
        else {
            name_dict['name'][i].visible = false;
            name_dict['legend'][i].visible = false;
        }
        
    }
""")

In [19]:
multi_choice.js_on_change('value', callback)

plot.legend.background_fill_alpha = 0.25
plot.add_tile(tile_provider)

layout = bokeh.layouts.row(
    plot, bokeh.models.Spacer(width=30), multi_choice
)
show(layout)

In [ ]:
bokeh.io.save(layout, "multiselect_map.html")

/opt/miniconda3/envs/basics/lib/python3.9/site-packages/bokeh/io/saving.py:142: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
/opt/miniconda3/envs/basics/lib/python3.9/site-packages/bokeh/io/saving.py:154: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")


'/Users/lianamerk/git/fermap/multiselect_map.html'